In [1]:
import hu_core_news_lg
import pandas as pd
import spacy
import os
from tqdm import tqdm

In [2]:
try:
    nlp = spacy.load("hu_core_news_lg")
except:
    print("A nagy modell nem található, a közepeset használjuk...")
    nlp = spacy.load("hu_core_news_md")

In [ ]:
nlp = hu_core_news_lg.load()

In [ ]:
RAW_CSV = r"../../data/propaganda_articles.csv"      # A nyers, eredeti fájl
PROCESSED_CSV = r"../../data/preprocessed_data.csv"

In [ ]:
csv_file = r"../../data/propaganda_articles.csv"

df = pd.read_csv(csv_file, encoding="utf-8-sig")

In [ ]:
df.sample()

In [ ]:
df.label.value_counts()

# Preprocessing

In [ ]:
def spacy_preprocess_pipe(texts, batch_size=50):
    """
    Gyorsabb verzió nlp.pipe használatával és tqdm folyamatjelzővel.
    """
    cleaned_texts = []
    
    # Itt a lényeg: a tqdm-be csomagoljuk az nlp.pipe-ot.
    # A 'total=len(texts)' fontos, hogy tudja, mennyi a 100%.
    for doc in tqdm(nlp.pipe(texts, batch_size=batch_size, disable=["ner", "parser"]), 
                    total=len(texts), 
                    desc="Szöveg előkészítése"):
        
        tokens = []
        for token in doc:
            if token.is_stop or token.is_punct:
                continue
            if token.like_num:
                tokens.append("NUM")
            else:
                tokens.append(token.lemma_)
        
        cleaned_texts.append(" ".join(tokens))
    
    return cleaned_texts

In [ ]:
def get_training_data():
    # 1. Betöltjük a nyers adatokat
    if not os.path.exists(RAW_CSV):
        raise FileNotFoundError(f"Nincs meg a forrásfájl: {RAW_CSV}")
        
    df_raw = pd.read_csv(RAW_CSV, encoding="utf-8-sig")
    df_raw = df_raw.dropna(subset=['title', 'text', 'url'])
    
    # 2. Megnézzük, van-e már mentett fájl
    if os.path.exists(PROCESSED_CSV):
        df_processed = pd.read_csv(PROCESSED_CSV, encoding="utf-8-sig")
        processed_urls = set(df_processed['url'])
        print(f"Betöltve {len(df_processed)} már feldolgozott cikk.")
    else:
        df_processed = pd.DataFrame(columns=['url', 'cleaned_text', 'label'])
        processed_urls = set()
        print("Még nincs feldolgozott fájl.")

    # 3. Kiszűrjük az újakat
    df_new = df_raw[~df_raw['url'].isin(processed_urls)].copy()

    if df_new.empty:
        print("Nincs új feldolgozandó cikk! A cache-ből dolgozunk.")
    else:
        print(f"Feldolgozás alatt: {len(df_new)} új cikk...")
        
        df_new['full_text'] = df_new['title'] + ": " + df_new['text']
        
        # --- ITT HÍVJUK MEG A TQDM-ES FÜGGVÉNYT ---
        df_new['cleaned_text'] = spacy_preprocess_pipe(df_new['full_text'].tolist())
        
        # 4. Mentés
        df_to_save = df_new[['url', 'cleaned_text', 'label']]
        header_needed = not os.path.exists(PROCESSED_CSV)
        
        df_to_save.to_csv(PROCESSED_CSV, mode='a', index=False, header=header_needed, encoding='utf-8-sig')
        
        # Összefűzés a memóriában
        df_processed = pd.concat([df_processed, df_to_save], ignore_index=True)
        print("Új adatok elmentve.")

    return df_processed['cleaned_text'], df_processed['label']

In [ ]:
def spacy_preprocess_one(text):
    """
    Egyetlen string preprocesszálása (inferencia/jóslás idejére).
    Ugyanazt a logikát követi, mint a spacy_preprocess_pipe.
    """
    # 1. Itt simán hívjuk az nlp-t, nem pipe-on keresztül
    # A disable itt is gyorsít, ha csak lemma kell
    doc = nlp(text, disable=["ner", "parser"])
    
    clean_tokens = []
    for token in doc:
        # Ugyanaz a szűrés
        if token.is_stop or token.is_punct:
            continue
        if token.like_num:
            clean_tokens.append("NUM")
        else:
            clean_tokens.append(token.lemma_)
    
    return " ".join(clean_tokens)

In [ ]:
x, y = get_training_data()
y = y.astype(int)
x.sample()

In [ ]:
y.value_counts()